In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
np.random.seed(42)

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [4]:
import tensorflow as tf
from tensorflow import keras

In [12]:
df1 = pd.read_csv('dc_signal_strength.csv')

In [13]:
df1.head()

,iPad ID,Access Point ID,Signal Strength
0,0452f3638d1a,b45d50f656d0,43
1,0452f3638d1a,b45d50f656d0,43
2,0452f3638d1a,b45d50f656d0,43
3,0452f3638d1a,b45d50f656d0,43
4,0452f3638d1a,b45d50f656d0,43


In [14]:
df2 = pd.read_csv('dc_ipads.csv')

In [15]:
df2.head()

,iPad Name,iPad ID,iPad x,iPad y,Floor
0,Big Sur,0452f3638d1a,3.0,49.2,3
1,Lake Shasta,0452f36388d5,10.0,40.8,3
2,Santa Beach,0452f3670522,20.3,40.8,3
3,Pismo Beach,0452f3640b3e,21.1,30.0,3
4,Malibu,0452f3696eed,24.0,40.8,3


In [16]:
df3 = pd.read_csv('dc_ap_locations.csv')

In [17]:
df3.head()

,Access Point ID,Access Point x,Access Point y
0,b45d50f656d0,2.725522,43.234657
1,b45d50f65730,25.640387,42.366589
2,b45d50f64a90,25.388924,18.573597
3,b45d50f66510,48.534828,35.651848
4,b45d50f80750,39.702026,50.005184


In [76]:
merge1 = pd.merge(df1, df2, how='outer', on='iPad ID')

In [78]:
merge1.tail(1)

,iPad ID,Access Point ID,Signal Strength,iPad Name,iPad x,iPad y,Floor
20684,0452f3640b3e,NaN,NaN,Pismo Beach,21.1,30.0,3


In [40]:
merge2 = pd.merge(merge1, df3, how='inner', on='Access Point ID')

In [41]:
merge2.shape

(20684, 9)

In [42]:
merge2.describe()

,Signal Strength,iPad x,iPad y,Floor,Access Point x,Access Point y
count,20684.000000,20684.000000,20684.000000,20684.0,20684.000000,20684.000000
mean,19.189083,58.279167,30.561594,3.0,55.499434,31.294047
std,10.037828,30.037094,10.428007,0.0,26.181892,13.342980
min,6.000000,3.000000,7.600000,3.0,2.725522,6.743395
25%,9.000000,24.000000,20.600000,3.0,39.361263,18.573597
50%,18.000000,67.000000,28.800000,3.0,59.658199,29.453739
75%,25.000000,85.800000,40.800000,3.0,77.493568,42.366589
max,45.000000,89.900000,49.200000,3.0,90.572845,55.907330


In [43]:
len(list(set(merge2['iPad ID'])))

16

In [44]:
len(list(set(merge2['Access Point ID'])))

37

Checking if 0452f360ef9f ipad id is in merge2

In [45]:
'0452f360ef9f' in merge2['iPad ID']

False

In [46]:
len(list(set(merge2['iPad Name'])))

16

In [47]:
from sklearn.tree import DecisionTreeRegressor

In [48]:
X = merge2[['Signal Strength', 'Access Point x', 'Access Point y']]
y1 = merge2[['iPad x']]
y2 = merge2[['iPad y']]

In [50]:
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)

In [51]:
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

In [52]:
df_reg = DecisionTreeRegressor(random_state=0)

In [54]:
df_reg.fit(X_train, y1_train)

DecisionTreeRegressor(random_state=0)

In [55]:
y1_pred = df_reg.predict(X_test)

In [56]:
y1_pred

array([67.        , 24.17382199, 67.        , ..., 24.        ,
       63.6       , 24.4       ])

In [57]:
from sklearn.metrics import mean_squared_error

In [58]:
mean_squared_error(y1_test, y1_pred)

22.698098897689274

In [59]:
df_reg.fit(X_train, y2_train)
y2_pred = df_reg.predict(X_test)
mean_squared_error(y2_test, y2_pred)

1.9577915742503142

In [60]:
from sklearn.ensemble import GradientBoostingRegressor

In [70]:
gb_reg = GradientBoostingRegressor(n_estimators=5000, max_depth=25)
gb_reg.fit(X_train, y1_train)
y1_pred_gb = gb_reg.predict(X_test)
mean_squared_error(y1_test, y1_pred_gb)

/Users/kalyaniasthana/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


22.698098897669556

In [72]:
lr = LinearRegression()
lr.fit(X_train, y1_train)
y1_pred_lr = lr.predict(X_test)
mean_squared_error(y1_test, y1_pred_lr)

367.700645442701

In [73]:
import xgboost as xgb

In [74]:
xgb_reg = xgb.XGBRegressor(n_estimators=1000, max_depth=25)
xgb_reg.fit(X_train, y1_train)
y1_pred_gb = xgb_reg.predict(X_test)
mean_squared_error(y1_test, y1_pred_gb)

22.698103990279005

In [75]:
df_reg.feature_importances_

array([0.17872573, 0.20969885, 0.61157542])